# Problem 2:
    
<blockquote> 1. Using sqoop copy data available in mysql products table to folder /user/cloudera/products on hdfs as text file. columns should be delimited by pipe '|'
move all the files from /user/cloudera/products folder to /user/cloudera/problem2/products folder

</blockquote>

<blockquote> 
2. move all the files from /user/cloudera/products folder to /user/cloudera/problem2/products folder

First we can go ahead and create a "problem2" folder under cloudera folder. "-p" flag is utilized to create recursive 
folder if they did not exits before.

</blockquote>

%%bash
hdfs dfs -mkdir -p /user/cloudera/problem2/


# We can now copy the entire products folder under the problem2 folder.

%%bash
hdfs dfs cp /user/cloudera/products /user/cloudera/problem2/

# Lets verify if the files are copied into the problem2 folder.

%%bash
hdfs dfs -ls /user/cloudera/problem2/products/


In [ ]:
<blockquote>3. Change permissions of all the files under /user/cloudera/problem2/products such that owner has read,write
and execute permissions, group has read and write permissions whereas 
others have just read and execute permissions
</blockquote>

hdfs dfs -chmod 765 /user/cloudera/problem2/products


7 - grants read, write, execute (111), 6 - grants read, write but not execute (110),
5 - grants read, no write, grants execute (101)

In [ ]:
4. read data in /user/cloudera/problem2/products and do the following operations using a) dataframes api b) spark sql c) RDDs aggregateByKey method. 

Your solution should have three sets of steps. Sort the resultant dataset by category id
    - filter such that your RDD\DF has products whose price is lesser than 100 USD
    - on the filtered data set find out the higest value in the product_price column under each category
    - on the filtered data set also find out total products under each category
    - on the filtered data set also find out the average price of the product under each category
    - on the filtered data set also find out the minimum price of the product under each category

In [ ]:
dataframes method:
    
Lets create a schema and utilize this for both DataFrame and SPARK SQL 

from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]'). \
        appName("Problem Scenario 3"). \
        enableHiveSupport().getOrCreate()
        
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import countDistinct

products_schmea = StructType([
    StructField("product_id", IntegerType(), True), \
    StructField("product_category_id", IntegerType(), True), \
    StructField("product_name", StringType(), True), \
    StructField("product_description", StringType(), True), \
    StructField("product_price", FloatType(), True), \
    StructField("product_image", StringType(), true)
])

products_df = spark.read.format("csv").schema(products_schema).mode("overwrite"). \
              load("/user/cloudera/problem2/products/part*")
    
prod_filter = products_df.where(col(product_price) < 100).groupBy("product_category_id")

prod_filter_max = prod_filter.groupBy("product_category_id").agg(max("product_price"))

prod_filter_max_ord = prod_filter_max.orderBy("product_category_id").show()

prod_filter_total = prod_filter.groupBy("product_category_id").agg(countDistinct("product_price"))

prod_filter_dis_ord = prod_filter_total.orderBy("product_category_id").show()

prod_filter_avg = prod_filter.groupBy("product_category_id").agg(avg("product_price"))

prod_filter_avg_ord = prod_filter_avg.orderBy("product_category_id").show()

prod_filter_min = prod_filter.groupBy("product_category_id").agg(min("product_price"))

prod_filter_min_ord = prod_filter_min.orderBy("product_category_id").show()

** SQL Method: **

spark.sql("USE cca_practice")

products_df.createOrReplaceTempView("products")

prod_sql_max = spark.sql("SELECT max(product_price) FROM products WHERE product_price < 100 \
          group by product_category_id order by product_catrgory_id").show()

prod_sql_total = spark.sql("SELECT count(distintct(product_price)) FROM products WHERE product_price < 100 \
          group by product_category_id order by product_catrgory_id").show()

prod_sql_avg = spark.sql("SELECT avg(product_price) FROM products WHERE product_price < 100 \
          group by product_category_id order by product_catrgory_id").show()

prod_sql_min = spark.sql("SELECT min(product_price) FROM products WHERE product_price < 100 \
          group by product_category_id order by product_catrgory_id").show()

5. store the result in avro file using snappy compression under these folders respectively
    - /user/cloudera/problem2/products/result-df
    - /user/cloudera/problem2/products/result-sql
    - /user/cloudera/problem2/products/result-rdd
    
    
If its the pyspark shell you are working, you need to declare the packages for avro at the start. 
Its dependent upon the Scala and Spark version you are working with.

pyspark --packages com.databricks:spark-avro_2.11:4.0.0
        
sqlContext.setConf("spark.sql.avro.compression.codec","codec")


#write the dataframe to the folder, in spark version > 2.3 Avro uses Snappy as default compression method.


prod_filter_max_ord.write.format("com.databricks.spark.avro").option("mode", "overwrite"). \
option("compression", "snappy").save("//user/cloudera/problem2/products/result-df/")


prod_sql_min.write.format("com.databricks.spark.avro").option("mode", "overwrite"). \
option("compression", "snappy").save("//user/cloudera/problem2/products/result-sql/")


# RDD Method:

'''from pyspark import SparkContext, SparkConf

sc = spark.sparkContext
    
    
prod_rdd = sc.textFile("/user/cloudera/problem2/products/part-00000")

prod_filt = prod_rdd.map(lambda x: ((x.split(",")[0], x.split(",")[4]), 1).filter(lambda z: (z[0] < 100.00, z[1]))'''

